In [1]:
%%javascript
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}setInterval(ClickConnect,60000)


<IPython.core.display.Javascript object>

# https://www.kaggle.com/devashishtiwari265/keyword-extraction-using-tf-idf/notebook

In [2]:
!mkdir -p dataset
%cd dataset
!wget "https://raw.githubusercontent.com/darkshadow013/Major-Project/master/dataset/papers.csv"
%cd ..

/content/dataset
--2021-04-28 07:37:59--  https://raw.githubusercontent.com/darkshadow013/Major-Project/master/dataset/papers.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9850371 (9.4M) [text/plain]
Saving to: ‘papers.csv’

papers.csv          100%[===================>]   9.39M  50.0MB/s    in 0.2s    

2021-04-28 07:38:00 (50.0 MB/s) - ‘papers.csv’ saved [9850371/9850371]

/content


In [3]:
import numpy as np
import pandas as pd
import os

os.system("python3 -m nltk.downloader wordnet")
os.system("python3 -m nltk.downloader stopwords")
os.system("mkdir -p ProcessingData")

df = pd.read_csv('dataset/papers.csv')
df = df.head(500)
import re
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

stop_words = set(stopwords.words('english'))
extra_words = ["fig", "show", "result", "large", "also", "one", "two", "three", "four", "five", "seven","eight","nine", "figure","image","sample","using"]
stop_words = list(stop_words.union(extra_words))

In [4]:
def pre_process_lemmatize(text):
    text=text.lower()
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    text=re.sub("(\\d|\\W)+"," ",text)
    text = text.split()
    text = [word for word in text if word not in stop_words]
    text = [word for word in text if len(word) >= 3]
    lmtzr = WordNetLemmatizer()
    text = [lmtzr.lemmatize(word) for word in text]
    return ' '.join(text)

In [5]:
if not os.path.exists("ProcessingData/docs.pkl"):
  docs = df['paper_text'].apply(lambda x:pre_process_lemmatize(x))
  docs.to_pickle("ProcessingData/docs.pkl")
else:
  docs = pd.read_pickle("ProcessingData/docs.pkl")
  print(len(docs))

In [6]:
import pickle
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_df=0.95,max_features=10000,ngram_range=(1,3))
if not os.path.exists("ProcessingData/cv.pkl"):
  cv.fit(docs)
  pickle.dump(cv, open("ProcessingData/cv.pkl", 'wb'))
else:
  cv = pickle.load(open("ProcessingData/cv.pkl", 'rb'))
  print(cv)

In [7]:
if not os.path.exists("ProcessingData/word_count_vector.pkl"):
  word_count_vector=cv.transform(docs)
  pickle.dump(word_count_vector, open("ProcessingData/word_count_vector.pkl", 'wb'))
else:
  word_count_vector = pickle.load(open("ProcessingData/word_count_vector.pkl", 'rb'))
  print(word_count_vector)

In [8]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer(smooth_idf=True,use_idf=True)

if not os.path.exists("ProcessingData/tfidf_transformer.pkl"):
  tfidf_transformer.fit(word_count_vector)
  pickle.dump(tfidf_transformer, open("ProcessingData/tfidf_transformer.pkl", 'wb'))
else:
  tfidf_transformer = pickle.load(open("ProcessingData/tfidf_transformer.pkl", 'rb'))
  print(tfidf_transformer)

In [9]:
def sort_coo_matrix(input_coo_matrix):
    tuples = zip(input_coo_matrix.col, input_coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    sorted_items = sorted_items[:topn]
    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results


In [10]:
feature_names=cv.get_feature_names()

In [11]:
def get_keywords(idx, docs):
    tf_idf_vector=tfidf_transformer.transform(cv.transform([docs[idx]]))
    sorted_items=sort_coo_matrix(tf_idf_vector.tocoo())
    keywords=extract_topn_from_vector(feature_names,sorted_items,5)
    
    return keywords

def print_results(idx,keywords, df):
    # now print the results
    print("\n=====Title=====")
    print(df['title'][idx])
    #print("\n=====Abstract=====")
    #print(df['abstract'][idx])
    print("\n===Keywords===")
    for k in keywords:
        print(k)

In [12]:
keywordsMap = dict()

In [13]:
for idx in range(0,len(docs)):
  keywords=get_keywords(idx, docs)
  #print_results(idx,keywords, df)
  title = df['title'][idx]
  for k in keywords:
    if k not in keywordsMap:
      keywordsMap[k] = []
    keywordsMap[k].append(title)

In [14]:
import json
with open("keywordsMap.json", "w") as outfile:  
    json.dump(keywordsMap, outfile, indent = 4) 